In [1]:
import h5py
import scipy
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from scipy.ndimage.filters import gaussian_filter
from scipy import spatial
import json
import torch
from multiprocessing import Pool
from functools import partial
import cv2
import matplotlib.pyplot as plt

In [5]:
get_ipython().system_raw("unrar x /content/drive/MyDrive/MobileCount/VisDrone2019-DET-train.rar")

In [2]:
pip install pyunpack

In [3]:
pip install patool

     |████████████████████████████████| 77 kB 3.0 MB/s 


In [ ]:
from pyunpack import Archive
Archive("/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/Dataset originale VisDrone2020-CC.7z" ).extractall("/content")

In [ ]:
#!unzip "/content/drive/MyDrive/MobileCount/VisDrone2019-DET-val.zip" -d "/content/"

In [ ]:
import shutil
#!unzip "/content/drive/MyDrive/MobileCount/VisDrone2019-DET-test-dev.zip" -d "/content/"
#shutil.move("/content/annotations", "/content/VisDrone2019-DET-test-dev")
#shutil.move("/content/images", "/content/VisDrone2019-DET-test-dev")

In [ ]:
#!unzip "/content/drive/MyDrive/DroneCrowd_960x540_COLAB/visdrone.zip" -d "/content/"

In [ ]:
def gaussian_filter_density(gt,img_path):
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    print(pts)
    leafsize = 2048
    # build kdtree
    tree = spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.
        if gt_count > 1:
            #print(str(distances[i][1]) + "," + str(distances[i][2]) + ", "+ str(distances[i][3]))
            sigma = (distances[i][1] + distances[i][2] + distances[i][3]) * 0.1
            #print("sigma 1: " +str(sigma))
        else:
            sigma = np.average(np.array(gt.shape)) / 2. / 2.  # case: 1 point
            #print("sigma 2: " +str(sigma))

        # Se ci sono errori con il sigma infinito rimuovo quella immagine
        try:
          density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
        except:

          !rm {img_path}
          #!rm {img_path.replace('images','images_resized_1920x1080')}
          !rm {img_path.replace('images','annotations').replace('.jpg','.txt')}

          print('Non considerata: ' , img_path)
          with open('/content/scartati_test.txt','a') as f:
            f.write(img_path + '\n')

    
    print('done.')
    return density

In [6]:
def max_size(img_paths):
  max_x = 0
  max_y = 0
  for img_path in img_paths:
    img = plt.imread(img_path)
    print("Height image: " + str(img.shape[0]) + ", Width image: " + str(img.shape[1]))
    if img.shape[0] > max_y:
      max_y = img.shape[0]
    if img.shape[1] > max_x:
      max_x = img.shape[0]
  return max_x,max_y

In [7]:
def min_size(img_paths):
  min_x = 1920
  min_y = 1080
  for img_path in img_paths:
    img = plt.imread(img_path)
    print("Height image: " + str(img.shape[0]) + ", Width image: " + str(img.shape[1]))
    if img.shape[0] < min_y:
      max_y = img.shape[0]
    if img.shape[1] < min_x:
      max_x = img.shape[0]
  return max_x,max_y

In [ ]:
import time
import numpy as np

def process(img_paths):

  for img_path in img_paths:
    img = plt.imread(img_path)
    print(img_path)
    people_coordinate_list = []
    file_annota = open(img_path.replace('images','annotations').replace('.jpg','.txt'), "r")
    print(img_path.replace('images','annotations').replace('.jpg','.txt'))


    lines = file_annota.readlines()
    file_annota.close()

    for ligne in lines:
      line = ligne.strip()
      line = line.split(",")
      line = list(map(int, line))
      if line[5] == 1 or line[5] == 2:
        people_coordinate_list.append([line[0],line[1]])

    # Resize dell'immagine
    if img.shape[1] >= img.shape[0]:  # Se la larghezza è maggiore dell'altezza (rettangolo in orizzontale)

        # Normalizzazione immagine
        #rate_1 = 1920.0 / img.shape[1]
        #rate_2 = 1080.0 / img.shape[0]

        # Con la resize diventa (width,height)
        # (0,0) è la dimensione minima dell'immagine
        # fx,fy il fattore di scaling dell'immagine
        #img = cv2.resize(img, (0, 0), fx=rate_1, fy=rate_2)
        k = np.zeros((img.shape[0],img.shape[1]))
        #cv2.imwrite(img_path.replace("images","images_resized_1920x1080"), img)

       # for x in people_coordinate_list:
          #x[0] = int(x[0]*rate_1)
          #x[1] = int(x[1]*rate_2)


    elif img.shape[0] > img.shape[1]:  # Se l'altezza è maggiore della larghezza (rettangolo in verticale)

        # Normalizzazione immagine
        #rate_1 = 1920.0 / img.shape[0]
        #rate_2 = 1080.0 / img.shape[1]

        # Con la resize diventa (width,height)
        # (0,0) è la dimensione minima dell'immagine
        # fx,fy il fattore di scaling dell'immagine
        #img = cv2.resize(img, (0, 0), fx=rate_2, fy=rate_1)
        k = np.zeros((img.shape[0],img.shape[1]))
        #cv2.imwrite(img_path.replace("images","images_resized_1920x1080"), img)

        #for x in people_coordinate_list:
          #x[0] = int(x[0]*rate_2)
          #x[1] = int(x[1]*rate_1)

    print(str(k.shape[0]) + "," + str(k.shape[1]))
    for i in range(0, len(people_coordinate_list)):
          # Controllo se le coordinate della persona sono nelle dimensioni dell'immagine
          if int(people_coordinate_list[i][1]) < img.shape[0] and int(people_coordinate_list[i][0]) < img.shape[1]:
              # Metto 1 nella matrice di zeri che corrisponde alla posizione di una persona
              k[people_coordinate_list[i][1], people_coordinate_list[i][0]] = 1

    start_time = time.time()
    k = gaussian_filter_density(k,img_path)
    print("Tempo di esecuzione : %s secondi ---" % (time.time() - start_time))
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
          hf['density'] = k
  

In [10]:
import os.path
from os import path

if __name__ == '__main__':

    '''if os.path.exists('/content/VisDrone2019-DET-train/ground_truth') == False:
      os.mkdir('/content/VisDrone2019-DET-train/ground_truth')

    if os.path.exists('/content/VisDrone2019-DET-train/images_resized_1920x1080') == False:
      os.mkdir('/content/VisDrone2019-DET-train/images_resized_1920x1080')
'''
    # Path delle immagini
    img_paths = []
    for dirpath, dirs, files in os.walk(
            '/content/VisDrone2019-DET-train/images'):
        for filename in files:
            fname = os.path.join(dirpath, filename)
            if fname.endswith('.jpg'):
                img_paths.append(fname)

    # Path delle immagini CROWD COUNTING
    '''for dirpath, dirs, files in os.walk(
            '/content/VisDrone2020-CC/train/sequences'):
        for filename in files:
            fname = os.path.join(dirpath, filename)
            if fname.endswith('.jpg'):
                img_paths.append(fname)

    # Path delle immagini CROWD COUNTING
    for dirpath, dirs, files in os.walk(
            '/content/VisDrone2020-CC/val/sequences'):
        for filename in files:
            fname = os.path.join(dirpath, filename)
            if fname.endswith('.jpg'):
                img_paths.append(fname)'''

    #process(img_paths)
    #x,y = max_size(img_paths)
    #print("Height: " + str(y) + ", Width: " + str(x))
    x,y = min_size(img_paths)
    print("Height: " + str(y) + ", Width: " + str(x))

Output streaming troncato alle ultime 5000 righe.
Height image: 1080, Width image: 1920
Height image: 1050, Width image: 1400
Height image: 540, Width image: 960
Height image: 765, Width image: 1360
Height image: 1078, Width image: 1916
Height image: 1500, Width image: 2000
Height image: 765, Width image: 1360
Height image: 1050, Width image: 1400
Height image: 788, Width image: 1400
Height image: 1500, Width image: 2000
Height image: 1050, Width image: 1400
Height image: 765, Width image: 1360
Height image: 765, Width image: 1360
Height image: 1500, Width image: 2000
Height image: 788, Width image: 1400
Height image: 1050, Width image: 1400
Height image: 788, Width image: 1400
Height image: 1050, Width image: 1400
Height image: 765, Width image: 1360
Height image: 1078, Width image: 1916
Height image: 1500, Width image: 2000
Height image: 788, Width image: 1400
Height image: 1050, Width image: 1400
Height image: 765, Width image: 1360
Height image: 788, Width image: 1400
Height image:

In [ ]:
import matplotlib.pyplot as plt
import PIL.Image as Image

plt.imshow(Image.open('/content/VisDrone2019-DET-train/images/0000003_00231_d_0000016.jpg'))

In [ ]:
plt.imshow(Image.open('/content/VisDrone2019-DET-train/images/9999967_00000_d_0000058.jpg'))

In [ ]:
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

img_path = '/content/VisDrone2019-DET-train/images_resized_960x540/9999997_00000_d_0000034.jpg'
img= cv2.imread(img_path)
print(str(img.shape[0]) + ","+ str(img.shape[1]))
print(img_path)
annotation = img_path.replace('.jpg','')
pos = 0
for nbr in annotation:
  if nbr != '0':
    #print(nbr)
    annotation = annotation[pos:]
    break
  pos+=1
#print(annotation)
file_annota = open(img_path.replace('images_resized_960x540','annotations').replace('.jpg','.txt'), "r")
print(img_path.replace('images_resized_960x540','annotations').replace('.jpg','.txt'))
#plt.imshow(img)
lines = file_annota.readlines()
file_annota.close()
for ligne in lines:
  #print(ligne.strip())
  line = ligne.strip()
  line = line.split(",")
  line = list(map(int, line))
  #print(str(line[0]) + ","+ str(line[1]))
  #print("Resized: " + str(int(line[0]*rate_1)) + "," + str(int(line[1]*rate_2)))
  if line[5] ==  1 or line[5] == 2:
    #print(line)

    img = cv2.circle(img,(int(line[0]*0.68),int(line[1]*0.51)), 5, (0,0,255), -1)

  
print("IMAGESIZE: " + str(img.shape[0]) + ", "+ str(img.shape[1]))
cv2_imshow(img)

In [ ]:
from matplotlib import cm as CM

gt_file = h5py.File('/content/VisDrone2019-DET-train/ground_truth/9999997_00000_d_0000034.h5')
groundtruth = np.asarray(gt_file['density'])
plt.imshow(groundtruth,cmap=CM.jet)

**ZIP**

In [ ]:
#!zip -r /content/visdrone2019DET.zip /content/VisDrone2019-DET-train

In [ ]:
#!zip -r /content/visdrone2019DET_val.zip /content/VisDrone2019-DET-val

In [ ]:
!zip -r /content/visdrone2019DET_test.zip /content/VisDrone2019-DET-test-dev

**COPY**

In [ ]:
#!cp /content/visdrone2019DET.zip /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/

In [ ]:
#!cp /content/visdrone2019DET_val.zip /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/

In [ ]:
!cp /content/visdrone2019DET_test.zip /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/

In [ ]:
from google.colab import files
files.download("/content/visdrone.zip")

In [ ]:
!unzip "/content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/visdrone2019DET_val.zip" -d "/content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/val"

In [ ]:
!unzip "/content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/visdrone2019DET_test.zip" -d "/content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/test"

In [ ]:
%cp -av /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/val/images_resized_960x540/. /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/train_val/images_resized_960x540/

COPY IMAGES RESIZED

In [ ]:
%cp -av /content/VisDrone2019-DET-train/images_resized_1920x1080/. /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_1920x1080/train/images_resized_1920x1080

COPY GROUND TRUTH

In [ ]:
%cp -av /content/VisDrone2019-DET-train/ground_truth/. /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_1920x1080/train/ground_truth

In [ ]:
%cp -av /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/train_val/images_resized_960x540 /content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/train_val/images

In [ ]:
import shutil
import os
id = []
for filename in os.listdir('/content/drive/MyDrive/MobileCount/Generare_density/ObjectDetection/Visdrone2019DET/dataset_960x540/val/ground_truth'):
  #if filename.endswith("h5") and filename[0:7] not in id:
  print(filename)
    #id.append(filename[0:7])


In [ ]:
# Divide dataset visdrone in sottocartelle

import shutil
import os
id = []
for filename in os.listdir('/content/content/Visdrone_leggero/part_B/test_data/ground'):
  #if filename[0:7] not in id and filename.endswith("jpg"): 
    #print(filename)
    #id.append(filename[0:7])
    #if os.path.exists('/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/Visdrone2020CC/dataset_960x540/test/images_resized_1920x1080/' + filename[0:7]) == False:
      #os.mkdir('/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/Visdrone2020CC/dataset_960x540/test/images_resized_1920x1080/' + filename[0:7])
  shutil.move('/content/content/Visdrone_leggero/part_B/test_data/ground/' + filename, '/content/drive/MyDrive/MobileCount/Generare_density/CrowdCounting/Visdrone2020CC/dataset_960x540/test/images_resized_1920x1080/' + filename[0:7])
